# PR Review Agent - Interactive Demo

This notebook demonstrates how to instantiate and use the PR Review Agent.

## Prerequisites

1. **Environment Setup**: Ensure `src/pr_agent/.env` is configured with:
   - `GOOGLE_CLOUD_PROJECT` - Your GCP project ID
   - `GOOGLE_CLOUD_LOCATION` - GCP region (e.g., `europe-west2`)
   - `REPOSITORY` - Repo identifier (e.g., `owner/repo`)
   - `PR_NUMBER` - PR/MR number to review
   - `GH_TOKEN` or `GITLAB_TOKEN` - Authentication token

2. **Authentication**:
   - Google Cloud: `gcloud auth application-default login`
   - GitHub: `gh auth login` or set `GH_TOKEN`
   - GitLab: `glab auth login` or set `GITLAB_TOKEN`

3. **Virtual Environment**: Ensure you're running in the project's venv with all dependencies installed

## 1. Import Dependencies and Load Environment

In [ ]:
import os
import sys
from pathlib import Path

# Add src directory to path for imports
project_root = Path.cwd().parent
src_dir = project_root / "src"
sys.path.insert(0, str(src_dir))

# Load environment variables
from dotenv import load_dotenv

load_dotenv(src_dir / "pr_agent" / ".env")

os.environ["REPOSITORY"] = "darkin100/i-am-reviewed"
os.environ["PR_NUMBER"] = "6"
os.environ["GOOGLE_CLOUD_AGENT_ENGINE_ENABLE_TELEMETRY"] = "true"
os.environ["OTEL_INSTRUMENTATION_GENAI_CAPTURE_MESSAGE_CONTENT"] = "true"
os.environ["ENABLE_CLOUD_TRACE"] = "false"

print("Environment loaded")
print(f"Project: {os.getenv('GOOGLE_CLOUD_PROJECT')}")
print(f"Location: {os.getenv('GOOGLE_CLOUD_LOCATION')}")
print(f"Repository: {os.getenv('REPOSITORY')}")
print(f"PR/MR Number: {os.getenv('PR_NUMBER')}")

Execute Main function to process PR review 

In [ ]:
import sys

sys.argv = ["pr_agent.workflow", "--provider", "github"]

# Apply nest_asyncio to allow asyncio.run() in Jupyter's event loop
import nest_asyncio

nest_asyncio.apply()

from pr_agent.workflow import workflow

workflow()